In [ ]:
!pip install -q datasets transformers accelerate


In [ ]:
from huggingface_hub import login

login(token="token")


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("akhilchilumuru/storybook-illustrations")


In [ ]:
# Check the available splits
print(dataset)

# Access the first example in the training set
first_example = dataset['train'][0]
print(first_example)


In [ ]:
!pip install datasets torchvision accelerate transformers diffusers --upgrade


In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("akhilchilumuru/storybook-illustrations", split="train")
print(dataset[0])  # Preview one sample


In [8]:
import os
from PIL import Image

save_dir = "/content/narrative_diffusion_lora"
os.makedirs(save_dir, exist_ok=True)

for i, sample in enumerate(dataset):
    image = sample["image"]
    caption = sample["text"]

    # Convert to RGB if it's RGBA or P mode
    if image.mode != "RGB":
        image = image.convert("RGB")

    image_path = os.path.join(save_dir, f"image_{i}.jpg")
    caption_path = os.path.join(save_dir, f"image_{i}.txt")

    image.save(image_path)
    with open(caption_path, "w") as f:
        f.write(caption)

    if i == 199:  # Limit for quick testing
        break


In [ ]:
!git clone https://github.com/kohya-ss/sd-scripts.git


In [10]:
%cd sd-scripts


/content/sd-scripts


In [ ]:
!pip install voluptuous


In [ ]:
import os

folder_path = "/content/narrative_diffusion_lora"

for filename in os.listdir(folder_path):
    if "_" in filename:
        new_name = filename.replace("_", "")
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))

print("All filenames updated successfully!")


In [ ]:
import os
import shutil

# Parameters
source_folder = "/content/narrative_diffusion_lora"
repeat_count = 400
character_name = "avatar"

# Destination path
destination_folder = f"/content/narrative_diffusion_lora/{repeat_count}_{character_name}"
os.makedirs(destination_folder, exist_ok=True)

# Move all .jpg and .txt files from source to destination
moved = 0
for filename in os.listdir(source_folder):
    if filename.endswith(".jpg") or filename.endswith(".txt"):
        src_path = os.path.join(source_folder, filename)
        dst_path = os.path.join(destination_folder, filename)
        shutil.move(src_path, dst_path)
        moved += 1

print(f"Moved {moved} files to: {destination_folder}")


In [ ]:
import subprocess

command = [
    "accelerate", "launch", "train_network.py",
    "--pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5",
    "--train_data_dir=/content/narrative_diffusion_lora",
    "--output_dir=/content/narrative_diffusion_lora_output",
    "--logging_dir=/content/narrative_diffusion_logs",
    "--resolution=512,512",
    "--train_batch_size=1",
    "--max_train_steps=500",
    "--save_model_as=safetensors",
    "--learning_rate=1e-4",
    "--mixed_precision=fp16",
    "--caption_extension=.txt",
    "--network_module=networks.lora",
    "--network_dim=16",
    "--output_name=narrative-diffusion-lora",
    "--cache_latents",
    "--prior_loss_weight=1.0"
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end='')


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from IPython.display import display

# Load base model
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# Load and fuse LoRA weights
pipe.load_lora_weights("/content/narrative_diffusion_lora_output", weight_name="narrative-diffusion-lora.safetensors")
pipe.fuse_lora()

# Test prompt
prompt = "Luna a curious girl with silver hair and a blue hooded cloak"
image = pipe(prompt).images[0]

# Display image
display(image)

